In [1]:
from scipy.stats import kendalltau

from scipy.optimize import linear_sum_assignment

In [2]:
import numpy as np
from matplotlib import pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from IPython import display

from scipy.spatial.transform import Rotation as R

import math
from copy import copy
import glob
import random

In [3]:
flist = sorted(glob.glob('../../demos/perms/order*'))
random.shuffle(flist)

In [4]:
# Load 300 examples
obj_list = []
im_list = []
for i,f in enumerate(flist[0:300]):
    run = int(f.split('_')[1][:-4])

    im = np.load('../../demos/perms/ims_%04d.npy'%run)
    obj_ids = np.load('../../demos/perms/order_%04d.npy'%run)
    
    obj_list.append(obj_ids)
    im_list.append(im)

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
device = torch.device("cuda:1")

In [6]:
# Set up pytorch dataloader
class Sampler(Dataset):
    
    def __init__(self, ims, actions):
        
        self.ims = torch.FloatTensor(ims.astype('float')).to(device)
        self.actions = torch.LongTensor(actions.astype('int')).to(device)
        
        
    def __len__(self):
        
        return self.ims.shape[0]
    
    def __getitem__(self, index):
        
        im = self.ims[index,:,:,:]
        actions = self.actions[index,:]
        return im, actions


In [7]:
dataset = Sampler(np.swapaxes(np.stack(im_list),1,3),np.stack(obj_list))

train_dataset,test_dataset = torch.utils.data.random_split(dataset, [100,len(im_list)-100])

RuntimeError: CUDA error: out of memory

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class BCNet(nn.Module):

    def __init__(self, latent_dim=16, image_channels=3, K=6):
        super(BCNet, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            Flatten(),
            nn.Linear(4096, latent_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        
        # Sinkhorn params
        self.latent_dim = latent_dim
        self.K = K
        
        self.criterion = nn.CrossEntropyLoss()

        self.fc = nn.Sequential(nn.Linear(self.latent_dim, self.latent_dim),
                                nn.ReLU(),
                        nn.Linear(self.latent_dim, K*K))
    
    def forward(self, im):
        
        latent = self.encoder(im)
        y = self.fc(latent)
        logits = torch.nn.functional.softmax(y.reshape(-1,self.K,self.K),dim=-1)
        
        return logits
    
    def loss(self, seq, im):
        
        seq_logits = self.forward(im)
    
        return self.criterion(seq_logits.view(-1,self.K),seq.view(-1)), seq_logits

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
bc = BCNet(latent_dim=128, image_channels=3, K=6)
bc.to(device)
optimizer = torch.optim.Adam(bc.parameters(), lr=3e-4)

In [ ]:
plt.figure(figsize=(15,5))

n_epochs = 10000
losses = []
val_losses = []
for j in range(n_epochs):
    
    batch_losses = []
    for im, seq in train_loader:
    
        loss, seq_pred = bc.loss(seq, im)
 
        batch_losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    if j % 10 == 0:
        losses.append(np.mean(batch_losses))

        batch_val_losses = []
        for im_t,seq_t in test_loader:
            loss, seq_pred_t = bc.loss(seq_t, im_t)
            batch_val_losses.append(loss.item())
        val_losses.append(np.mean(batch_val_losses)) 

        plt.clf()
        plt.cla()
        plt.subplot(1,3,1)
        plt.plot(losses,alpha=0.5)
        plt.plot(val_losses,alpha=0.5)
        plt.title('Loss')


        plt.subplot(3,3,2)
        seq_p = np.argmax(seq_pred[0,:,:].cpu().detach().numpy(),-1)
        plt.imshow(seq_p.reshape(1,-1))
        plt.title('Re-shuffled seq')

        plt.subplot(3,3,8)
        plt.imshow(seq[0,:].cpu().detach().numpy().reshape(1,-1))
        plt.title('Original seq')

        plt.subplot(1,3,3)
        plt.imshow(np.swapaxes(im[0,:,:,:].cpu().detach().numpy(),0,2))


        display.clear_output(wait=True)
        display.display(plt.gcf())


In [ ]:
bc.eval()

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
tau_list = []
for im,seq in test_loader:
    seq_pred = bc(im)
    obj_ids = np.argmax(seq_pred[0,:,:].cpu().detach().numpy(),-1)
    tau, _ = kendalltau(obj_ids, seq[0,:].cpu().numpy())
    tau_list.append(tau)

tau_list_hung = []
for im,seq in test_loader:
    seq_pred = bc(im)
    _,obj_ids = linear_sum_assignment(1.0-seq_pred[0,:,:].cpu().detach().numpy())
    tau, _ = kendalltau(obj_ids, seq[0,:].cpu().numpy())
    tau_list_hung.append(tau)

plt.plot(tau_list)
plt.plot(tau_list_hung)
plt.ylabel('Kendall\'s $\\tau$')
plt.xlabel('Demonstration')
plt.show()

In [ ]:
from pyrep import PyRep
from pyrep.robots.arms.panda import Panda
from pyrep.robots.end_effectors.panda_gripper import PandaGripper
import numpy as np
from matplotlib import pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from IPython import display

from pyrep.objects.shape import Shape
from pyrep.const import PrimitiveShape

from scipy.spatial.transform import Rotation as R

import math
from copy import copy

In [ ]:
pr = PyRep()
pr.launch('../../assets/scene_panda.ttt', headless=False)

In [ ]:
agent = Panda()
gripper = PandaGripper()

In [ ]:
home_pos = agent.get_tip().get_position()
home_orient = agent.get_tip().get_orientation()

In [ ]:
def grasp(grip=False):
    if grip:
        pos = 0.1
    else:
        pos = 0.9
    actuated = False
    ims = []
    states = []
    while not actuated:
        actuated = gripper.actuate(pos,0.1)
        im0,im1 = get_image()
        ims.append((im0,im1))
        states.append(agent.get_tip().get_pose())
    return ims,states

In [ ]:
def move_above_object(object_name='',offset=0.05):
    pos = agent.get_object(object_name).get_position()
    pos[2] = pos[2] + offset
    orient = [-np.pi,0,np.pi/2]

    path = agent.get_path(position=pos,euler=orient)

    done = False
    ims = []
    states = []
    while not done:
        done = path.step()
        im0,im1 = get_image()
        ims.append((im0,im1))
        states.append(agent.get_tip().get_pose())
    return ims,states

In [ ]:
def get_image():
    cam = agent.get_object('Vision_sensor_front')
    im0 = cam.capture_rgb()
    cam1 = agent.get_object('Vision_sensor')
    im1 = cam1.capture_rgb()
    pr.step()
    return im0, im1

In [ ]:
# Set up a dataloader for testing
tl = iter(train_loader)

# Run this repeatedly to see how the action sequencing works for different images
im,seq = next(tl)
seq_pred = bc(im)
_,obj_ids = linear_sum_assignment(1.0-seq_pred[0,:,:].detach().numpy())

pr.start()

grasp(grip=False)
gripper.release()
ims = []
states = []
for j in range(1,len(obj_ids)):
    object_name = 'Cuboid%d'%obj_ids[j]
    i,s = move_above_object(object_name,offset=0.08)
    ims = ims + i
    states = states + s
    i,s = move_above_object(object_name,offset=0)
    ims = ims + i
    states = states + s
    i,s = grasp(grip=True)
    ims = ims + i
    states = states + s
    gripper.grasp(agent.get_object(object_name))
    i,s = move_above_object(object_name,offset=0.08)
    ims = ims + i
    states = states + s
    object_name = 'Cuboid%d'%obj_ids[j-1]
    i,s = move_above_object(object_name,offset=0.15)
    ims = ims + i
    states = states + s
    i,s = move_above_object(object_name,offset=0.05)
    ims = ims + i
    states = states + s
    i,s = grasp(grip=False)
    ims = ims + i
    states = states + s
    gripper.release()
    i,s = move_above_object(object_name,offset=0.2)
    ims = ims + i
    states = states + s
    #clear_view()
    
    plt.cla()
    plt.clf()
    plt.subplot(1,3,1)
    plt.imshow(ims[-1][0])
    plt.title('Actual')
    plt.subplot(1,3,2)
    plt.imshow(ims[-1][1])
    plt.subplot(1,3,3)
    plt.imshow(np.swapaxes(im.numpy()[0,:,:],2,0))
    plt.title('Target')
    display.clear_output(wait=True)
    display.display(plt.gcf())

pr.stop()

In [ ]:
pr.shutdown()